In [33]:
from selenium import webdriver
import time
import datetime
from selenium.webdriver.common.by import By
import pickle
from tqdm import tqdm
from preparing import preparing_cite
import pandas as pd
from statistic import statistics_info
# agents options
statistics = True
import random
from IPython.display import clear_output
import os

In [3]:
# options
options = webdriver.ChromeOptions()

# user-agent
options.add_argument("user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0")
# for ChromeDriver version 79.0.3945.16 or over
options.add_argument("--disable-blink-features=AutomationControlled")

driver = webdriver.Chrome(executable_path=r'C:\Users\user\Project-Cars\drivers\Chromedriver\chromedriver.exe',
                          options=options)

cite = 'https://auto.ru/'


driver.get(cite)
driver.find_element(by = By.XPATH, value = '//*[@id="root"]/div/div/form/div[2]/div/div/div[1]/input').click()
time.sleep(15)
for cookie in pickle.load(open(f"auto_cookies", 'rb')):
    driver.add_cookie(cookie)
driver.refresh()
# driver.find_element(by = By.XPATH, value = '//*[@id="root"]/div/div/form/div[2]/div/div/div[1]/input').click()
# time.sleep(15)
marks_tidy = driver.find_elements(by = By.CLASS_NAME, value = 'IndexMarks__item')
count_tidy = driver.find_elements(by = By.CLASS_NAME, value = 'IndexMarks__item-count')
marks = []
counts = []
city = driver.find_element(by = By.CLASS_NAME, value = 'GeoSelect__titleShrinker-wjCdV').text.split(' ')[0]
for j in tqdm(range(len(marks_tidy)), desc = 'Работаю с марками'):
    marks.append(marks_tidy[j].text.split('\n')[0])

C:\Users\user\AppData\Local\Temp\ipykernel_11664\1067538451.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'C:\Users\user\Project-Cars\drivers\Chromedriver\chromedriver.exe',
Работаю с марками: 100%|██████████| 34/34 [00:00<00:00, 62.90it/s]


In [5]:
statistics = True
if statistics == True:
    for i in tqdm(range(len(count_tidy)), desc='Считаю статистики'):
        counts.append(count_tidy[i].text.split('\n')[0])
    dict_for_statistic = {
        'Marks': marks,
        'Counts': counts,
        'City': [city] * len(marks)
    }
    assert len(marks) == len(counts), 'Ошибка в реализации или в выдаче сайте'
    pd.DataFrame(dict_for_statistic).to_csv(f'Статистика_{city}.csv')

Считаю статистики: 100%|██████████| 34/34 [00:00<00:00, 76.66it/s]


In [34]:
try:
    marks_done = list(pd.read_csv('All.csv')['Mark'].unique())
except:
    marks_done = []

In [36]:
p = 0
for z in tqdm(range(len(marks)), desc = 'Парсирую юрл марок (вообще)'):
    if marks[z] in marks_done:
        continue
    try: marks_tidy[z].click()
    except:
        marks_tidy = driver.find_elements(by = By.CLASS_NAME, value = 'IndexMarks__item')
        marks_tidy[z].click()
    link = []
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.randint(4, 7))
    # try:
    #     # page_max = driver.find_element(by = By.XPATH, value = '//*[@id="listing-filters"]/div[10]/div/span/a[11]/span')\
    #     # .text.split('\n')[-1]
    #     page_max = driver.find_element(by = By.XPATH, value = '//*[@id="listing-filters"]/div[10]/div/span').text[-1]
    # except:
    #     page_max = driver.find_element(by = By.XPATH, value = '//*[@id="lmcQaMxB6xjLkJq45u4ay"]/div[10]/div/span/a[11]/span')\
    #             .text.split('\n')[-1]
    # print(f'Всего будет {page_max} страниц')
    print('Готов')
    for i in tqdm(range(2, int(40) + 1), desc = f'Страницы'):
        p += 1
        time.sleep(random.randint(4, 7))
        urls = driver.find_elements(by = By.CLASS_NAME, value = 'LazyImage__image')
        for url in urls:
            link.append(url.get_attribute('src'))
            # except:
            #     time.sleep(5)
            #     urls = driver.find_elements(by = By.CLASS_NAME, value = 'LazyImage__image')
            #     for url in urls:
            #         link.append(url.get_attribute('src'))
            #    break
        if p % 5 == 0:
            d = {'Urls' : link,
                 'Mark' : [marks[z]] * len(link)}
            urls_gap = pd.DataFrame(d)
            urls_gap.to_csv(f'Urls_gap_{marks[z]}.csv')
        # if page_max == 1:
        #     break
        try: driver.find_element(by = By.LINK_TEXT, value = f'{i}').click()
        except:
            break
        if 0.5 > random.uniform(0, 1):
            time.sleep(random.randint(3, 5))
    driver.get(cite)
    clear_output()

Парсирую юрл марок (вообще): 100%|██████████| 34/34 [15:04<00:00, 26.61s/it]


In [37]:
pd_all = pd.DataFrame()
for mark in marks:
    try:
        pd_all = pd_all.append(pd.read_csv(f'Urls_gap_{mark}.csv', index_col = 0))
        os.remove(f'Urls_gap_{mark}.csv')
        clear_output()
    except:
        continue
pd_all = pd_all.reset_index(drop=True)
pd_all_past = pd.read_csv('All.csv', index_col = 0)
pd_all = pd_all_past.append(pd_all)
pd_all.to_csv('All.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_19436\3357959652.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pd_all = pd_all_past.append(pd_all)


Напишем параллельно...